1. Load raster from EE
2. Load style file from drive
3. Generate STAC item   
    a. generate bounding box  
    b. start and end date from file  
    c. Create COG  
    d. Generate thumbnail and save it  
   

In [ ]:
# !pip install pystac[validation]

In [ ]:
# ! pip install pystac
# ! pip install rasterio
# ! pip install shapely

In [33]:
# !pip install python-dateutil

In [46]:
import os
import datetime
import rasterio
import urllib.request
import pystac
import dateutil

from shapely.geometry import Polygon, mapping
from tempfile import TemporaryDirectory

from pystac.extensions.eo import Band, EOExtension
from pystac.extensions.view import ViewExtension
from pystac.extensions.projection import ProjectionExtension
import pystac.extensions.raster 

In [2]:
os.getcwd()

'/home/nirzaree/Code/GitClones/STAC-spec'

In [ ]:
# Set the temporary directory to store source data
# tmp_dir = TemporaryDirectory()
img_path = os.path.join('data',
                        'saraikela-kharsawan_gobindpur_2019-07-01_2020-06-30_LULCmap_10m.tif')

In [41]:
img_path

'data/saraikela-kharsawan_gobindpur_2019-07-01_2020-06-30_LULCmap_10m.tif'

In [ ]:
def get_bbox_and_footprint(raster):
    with rasterio.open(raster) as r:
        crs = r.crs
        res = r.res
        transform = r.transform
        linear_units = crs.linear_units
        bounds = r.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])       
        tags = r.tags()
        date_str = tags.get("TIFFTAG_DATETIME")
        print(date_str)
        return (bbox, mapping(footprint),crs,res,transform,linear_units)

In [51]:
bbox, footprint,crs,res,datatype,transform,linear_units = get_bbox_and_footprint(img_path)

AttributeError: 'DatasetReader' object has no attribute 'dtype'

In [18]:
filename = os.path.basename(img_path)
filename

'saraikela-kharsawan_gobindpur_2019-07-01_2020-06-30_LULCmap_10m.tif'

In [19]:
district,block,start_date,end_date,layer_description,scale = filename.split('_')

In [31]:
print("district =",district,
      ",\nblock = ",block,
      "\nstart_date = ",start_date,
      "\nend_date = ",end_date,
      "\nlayer_description = ",layer_description,
      "\nscale = ",scale
      )

district = saraikela-kharsawan ,
block =  gobindpur 
start_date =  2019-07-01 
end_date =  2020-06-30 
layer_description =  LULCmap 
scale =  10m.tif


In [16]:
start_date

'2019-07-01'

In [17]:
end_date

'2020-06-30'

In [35]:
def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        dateutil.parser.parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [ ]:
def get_start_date_and_end_date(filename):
    try:
        district,block,start_date,end_date,layer_description,scale = filename.split('_')
        if (is_date(start_date) & (is_date(end_date))):
            
    except Exception as e:
        print("File format not as expected in {filename}")

In [49]:
from pystac.extensions.raster import DataType

In [48]:
item = pystac.Item(id='raster-poc',
                      geometry=footprint,
                      bbox=bbox,
                      datetime=datetime.datetime.now(),
                    #   start_datetime = start_date,
                    #   end_datetime=end_date,
                      properties={                        
                      })

In [ ]:
  with rasterio.open('your_raster_file.tif') as src:
        data_type = src.dtype
        print(f"The data type of the raster is: {data_type}")

In [ ]:
# item.validate()

In [ ]:
with rasterio.open(img_path) as r:
    img = r.read()

In [ ]:
# !pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
r.count

In [ ]:
img2 = img.reshape(img.shape[1],img.shape[2])

In [ ]:
img2.shape

In [ ]:
vis_params = {
    'min': 0,
    'max': 12,
    'palette': ['#000000',  # 0 Black- background
            '#ff0000',   # 1 Red- builtup
            '#74ccf4', # 2 Light Blue- kharif water
            '#1ca3ec', # 3 Blue- kharif and rabi water
            '#0f5e9c', # 4 Dark Blue- kharif and rabi and zaid water
            '#f1c232', # 5 Yellow- croplands
            '#38761d', # 6 Dark Green- Tree/Forests
            '#A9A9A9', # 7 Gray- barren lands
            '#f1c232', # 8 Yellow- Single Kharif Cropping
            '#f59d22', # 9 Mustard- Single Non-Kharif Cropping
            '#e68600', # 10 Orange- Double Cropping
            '#b3561d', # 11 Brown- Triple Cropping
            '#c39797' # 12 Mauve- Shrubs_Scrubs
            ]
    }

In [ ]:
import matplotlib

In [ ]:
vis_params['palette']

In [ ]:
cmap = matplotlib.colors.ListedColormap(vis_params['palette'])
norm = matplotlib.colors.BoundaryNorm(range(len(vis_params['palette'])), cmap.N)


In [ ]:
plt.imshow(img2,interpolation='none', cmap=cmap, norm=norm) #https://stackoverflow.com/questions/32766062/how-to-determine-the-colours-when-using-matplotlib-pyplot-imshow

In [ ]:
import ee

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='ee-corestackdev')

In [ ]:
gobindpur_2023_lulc = ee.Image('projects/ee-corestackdev/assets/apps/mws/jharkhand/saraikela-kharsawan/gobindpur/saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m')

In [ ]:
gobindpur_2023_lulc = ee.Image('projects/ee-corestackdev/assets/apps/mws/jharkhand/dumka/masalia/dumka_masalia_2023-07-01_2024-06-30_LULCmap_10m')

In [ ]:
gobindpur_2023_lulc

In [ ]:
import geemap

In [ ]:
vis_params = {
    'min': 0,
    'max': 12,
    'palette': ['#000000',  # 0 Black- background
            '#ff0000',   # 1 Red- builtup
            '#74ccf4', # 2 Light Blue- kharif water
            '#1ca3ec', # 3 Blue- kharif and rabi water
            '#0f5e9c', # 4 Dark Blue- kharif and rabi and zaid water
            '#f1c232', # 5 Yellow- croplands
            '#38761d', # 6 Dark Green- Tree/Forests
            '#A9A9A9', # 7 Gray- barren lands
            '#f1c232', # 8 Yellow- Single Kharif Cropping
            '#f59d22', # 9 Mustard- Single Non-Kharif Cropping
            '#e68600', # 10 Orange- Double Cropping
            '#b3561d', # 11 Brown- Triple Cropping
            '#c39797' # 12 Mauve- Shrubs_Scrubs
            ]
    }

In [ ]:
m = geemap.Map()
m.add_layer(gobindpur_2023_lulc,vis_params)
m

In [ ]:
m

In [ ]:
import PIL

In [ ]:
size = (128,128)

In [ ]:
image = Image.fromarray(img2)
image.thumbnail(size)
image.save(thumbnail_path)

In [ ]:
def displayMap(roi_boundary, image):
    centroid = roi_boundary.geometry().centroid()
    coordinates = centroid.coordinates()
    centerLat = coordinates.get(1).getInfo()
    centerLon = coordinates.get(0).getInfo()
    mapObj = folium.Map(width='100%', height='100%', location=[centerLat, centerLon], zoom_start=50)

    # Sattelite image visual parameters
    vis_params = {
        'min': 0,
        'max': 12,
        'palette': ['#000000',  # 0 Black- background
                '#ff0000',   # 1 Red- builtup
                '#74ccf4', # 2 Light Blue- kharif water
                '#1ca3ec', # 3 Blue- kharif and rabi water
                '#0f5e9c', # 4 Dark Blue- kharif and rabi and zaid water
                '#f1c232', # 5 Yellow- croplands
                '#38761d', # 6 Dark Green- Tree/Forests
                '#A9A9A9', # 7 Gray- barren lands
                '#f1c232', # 8 Yellow- Single Kharif Cropping
                '#f59d22', # 9 Mustard- Single Non-Kharif Cropping
                '#e68600', # 10 Orange- Double Cropping
                '#b3561d', # 11 Brown- Triple Cropping
                '#c39797' # 12 Mauve- Shrubs_Scrubs
                ]
        }

    map_id_dict = ee.Image(image).getMapId(vis_params)

    folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='Esri',
            name='Esri Satellite',
            overlay=True,
            control=True
        ).add_to(mapObj)

    folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = 'Sentinel 2 image',
                    overlay = True,
                    control = True
                    ).add_to(mapObj)

    roi_geojson = folium.GeoJson(data=roi_boundary.getInfo(), name='ROI Boundary', style_function=lambda x: {'color':'black', 'fillColor': 'transparent', 'fillOpacity': 0})
    # roi_geojson.add_to(mapObj)

    # Add legend
    legend_html = '''
    <div style="position: fixed; bottom: 200px; right: 100px; z-index:1000; border:2px solid black; background-color:white;
        border-radius:5px; padding:10px; font-size:14px;">
        <p style="font-weight: bold;">Legend</p>
        <p><span style="color: black; font-size: 16px;">■</span> ROI Boundary </p>
        <p><span style="color: #ff0000; font-size: 16px;">■</span> Built-up </p>
        <p><span style="color: #74ccf4; font-size: 16px;">■</span> Water in Kharif </p>
        <p><span style="color: #1ca3ec; font-size: 16px;">■</span> Water in Kharif and Rabi </p>
        <p><span style="color: #0f5e9c; font-size: 16px;">■</span> Water in Kharif, Rabi and Zaid </p>
        <p><span style="color: #38761d; font-size: 16px;">■</span> Tree/Forests </p>
        <p><span style="color: #A9A9A9; font-size: 16px;">■</span> Barren Lands </p>
        <p><span style="color: #f1c232; font-size: 16px;">■</span> Single-Kharif Cropping </p>
        <p><span style="color: #f59d22; font-size: 16px;">■</span> Single Non-Kharif Cropping </p>
        <p><span style="color: #e68600; font-size: 16px;">■</span> Double Cropping </p>
        <p><span style="color: #b3561d; font-size: 16px;">■</span> Triple Cropping </p>
        <p><span style="color: #c39797; font-size: 16px;">■</span> Shrubs and Scrubs </p>
    </div>
    '''
    # mapObj.get_root().html.add_child(folium.Element(legend_html))

    mapObj.add_child(folium.LayerControl())
    display(mapObj)